# Домашнее задание к занятию
## "Классификация: Логистическая регрессия и SVM"

Имеются данные adult.csv (см. в материалах к занятию).

Целевой переменной является уровень дохода income (крайний правый столбец).

Описание признаков можно найти по ссылке www.cs.toronto.edu...etail.html

Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.

Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.

In [1]:
# Загружаем основные библиотеки
import numpy as np
import pandas as pd 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

In [2]:
# Загружаем датасет
adult_data = pd.read_csv('adult.csv', na_values='?')
adult_data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


Описание признаков:
- age: возраст (непрерывная величина).
- workclass: класс работы: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- fnlwgt: вес выборки (непрерывная величина).
- education: образование: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- education-num: образование (непрерывная величина).
- marital-status: семейное положение: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- occupation: профессия: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- relationship: отношения: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- race: раса: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- gender: пол: Female, Male.
- capital-gain: прирост капитала (непрерывная величина).
- capital-loss: потеря капитала (непрерывная величина).
- hours-per-week: количество рабочих часов в неделю (непрерывная величина).
- native-country: родная страна: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
- income: уровень дохода: >50K, <=50K

In [3]:
# Получаем основную информацию о данных в датасете
adult_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        46043 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       46033 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   47985 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


Из полученных данных видим, что типы объектов соответствуют типам данных в колонках.

В колонках workclass, occupation, native-country отсутствует часть данных, составляющих 5.7%, 5.8%, 1,8%, соответственно. 

In [4]:
adult_data[pd.isnull(adult_data['workclass'])].head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K
6,29,NaN,227026,HS-grad,9,Never-married,NaN,Unmarried,Black,Male,0,0,40,United-States,<=50K
13,58,NaN,299831,HS-grad,9,Married-civ-spouse,NaN,Husband,White,Male,0,0,35,United-States,<=50K
22,72,NaN,132015,7th-8th,4,Divorced,NaN,Not-in-family,White,Female,0,0,6,United-States,<=50K
35,65,NaN,191846,HS-grad,9,Married-civ-spouse,NaN,Husband,White,Male,0,0,40,United-States,<=50K


Видим, что в строках, где пропущено значение в колонке workclass пропущено и значение в колонке occupation. Таким образом, восстановить зависимость класса работы от профессии не представляется возможным. Дополнительно, в связи с тем, что работа с пропусками будет проводится в дальнейшем и в виду небольшого количества пропусков, принято решение об удалении строк с пропущенными значениями.

In [5]:
# Удаляем ячейки с пропущенными данными
adult_data = adult_data[pd.isnull(adult_data['workclass']) == 0 ]
adult_data = adult_data[pd.isnull(adult_data['occupation']) == 0 ]
adult_data = adult_data[pd.isnull(adult_data['native-country']) == 0 ]

In [6]:
# Получаем основную информацию о данных в откорректированном датасете
adult_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              45222 non-null  int64 
 1   workclass        45222 non-null  object
 2   fnlwgt           45222 non-null  int64 
 3   education        45222 non-null  object
 4   educational-num  45222 non-null  int64 
 5   marital-status   45222 non-null  object
 6   occupation       45222 non-null  object
 7   relationship     45222 non-null  object
 8   race             45222 non-null  object
 9   gender           45222 non-null  object
 10  capital-gain     45222 non-null  int64 
 11  capital-loss     45222 non-null  int64 
 12  hours-per-week   45222 non-null  int64 
 13  native-country   45222 non-null  object
 14  income           45222 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.5+ MB


In [7]:
# Определим распределение заполнения колонки education
adult_data['education'].value_counts()

HS-grad         14783
Some-college     9899
Bachelors        7570
Masters          2514
Assoc-voc        1959
11th             1619
Assoc-acdm       1507
10th             1223
7th-8th           823
Prof-school       785
9th               676
12th              577
Doctorate         544
5th-6th           449
1st-4th           222
Preschool          72
Name: education, dtype: int64

In [8]:
# Определим распределение заполнения колонки educational-num
adult_data['educational-num'].value_counts()

9     14783
10     9899
13     7570
14     2514
11     1959
7      1619
12     1507
6      1223
4       823
15      785
5       676
8       577
16      544
3       449
2       222
1        72
Name: educational-num, dtype: int64

На основе полученных результатов и данных, полученных при визуальном просмотре датасета, приходим к выводу, что столбцы education и educational-num содержат аналогичную информацию. Уровень образования является категориальным признаком, которые можно отсортировать по-порядко, что и отражено в колонке educational-num. Таким образом, в дальнейшем можно использовать только колонку educational-num.

In [9]:
# Удаляем колонку education
new_adult_data = adult_data.copy()
new_adult_data.pop('education')
new_adult_data.head()

,age,workclass,fnlwgt,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
5,34,Private,198693,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K


Для составления пресказания определим признаки, по которым будем осуществлять предсказание уровня дохода человека. Это колонки: age, workclass, educational-num, occupation, race, gender, capital-gain, capital-loss, hours-per-week.

Класс работы, профессию, расу невозможно отсортировать от большего к меньшему, таким образом для них воспользуемся get_dummies.

In [10]:
new_adult_data = pd.get_dummies(new_adult_data[['age', 'workclass', 'educational-num', 'occupation', 'race', 
                                                'gender', 'capital-gain', 'capital-loss', 'hours-per-week', 'income']], 
                                columns = ['workclass', 'occupation', 'race'])
new_adult_data.head()

,age,educational-num,gender,capital-gain,capital-loss,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,workclass_Private,...,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,25,7,Male,0,0,40,<=50K,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,38,9,Male,0,0,50,<=50K,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,28,12,Male,0,0,40,>50K,0,1,0,...,0,1,0,0,0,0,0,0,0,1
3,44,10,Male,7688,0,40,>50K,0,0,1,...,0,0,0,0,0,0,0,1,0,0
5,34,6,Male,0,0,30,<=50K,0,0,1,...,0,0,0,0,0,0,0,0,0,1


Столбцы gender и income переведем в значения 0 и 1

In [11]:
le = LabelEncoder()
le_ = LabelEncoder()
le.fit(new_adult_data['gender'])
le_.fit(new_adult_data['income'])
new_adult_data['gender'] = le.transform(new_adult_data['gender'])
new_adult_data['income'] = le_.transform(new_adult_data['income'])
new_adult_data.head()

,age,educational-num,gender,capital-gain,capital-loss,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,workclass_Private,...,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,25,7,1,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,38,9,1,0,0,50,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,28,12,1,0,0,40,1,0,1,0,...,0,1,0,0,0,0,0,0,0,1
3,44,10,1,7688,0,40,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
5,34,6,1,0,0,30,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [12]:
# Разделим целевую переменную и данные, по которым будет осуществляться предсказание
y = pd.Series(new_adult_data['income'])
new_adult_data.pop('income')
X = new_adult_data

In [13]:
# Разделим данные на обучающие и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Логистическая регрессия

In [14]:
# В качестве модели используем логистическую регресиию из scikit-learn
model_1 = LogisticRegression(max_iter=10000)

In [15]:
# Обучаем на тренировочной части датасета и строим предсказание
model_1.fit(X_train, y_train)
predictions = model_1.predict(X_test)

In [16]:
# Выведем качество полученной модели на тестовой выборке
model_1.score(X_test,y_test)

0.8257536669860691

In [17]:
# Аналогично для логистической регрессии с масштабированием
model_2 = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))
model_2.fit(X_train, y_train)
predictions = model_2.predict(X_test)
model_2.score(X_test,y_test)

0.8266381661384241

Качество у модели с масштабированием незначительно выше, требуется меньшее количество итераций.

## SVM

In [18]:
# Для модели без масштабирования
clf_1 = SVC(gamma='auto')
clf_1.fit(X_train, y_train)

SVC(gamma='auto')

In [19]:
# Выведем качество полученной модели на тестовой выборке
clf_1.score(X_test, y_test)

0.8482346871084249

In [21]:
# Для модели с масштаборованием
clf_2 = make_pipeline(StandardScaler(), SVC(gamma='auto')) 
clf_2.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [22]:
# Выведем качество полученной модели на тестовой выборке
clf_2.score(X_test, y_test)

0.8337141593572639

Качество модели SVM выше, чем у модели логистической регрессии. При масштабировании качество модели снизилось.